In [ ]:
import os
import numpy as np
dir_path = "/home/wouter/onesnake"
os.chdir(dir_path)
import matplotlib.pyplot as plt
from msmtools.analysis import (is_transition_matrix, is_connected, mfpt, 
                               stationary_distribution, is_reversible,
                               committor)
from msmtools.flux import tpt
from msmtools.estimation import connected_sets
from pprint import pprint 

%matplotlib qt
%load_ext autoreload

def pathens_to_dict(fn):
    pdict = {}
    with open(fn, "r") as f:
        for line in f:
            data = line.split()
            p = float(data[16])
            # w = float(data[17]) BEFORE
            w = 1.
            ptype = data[18]
            # check whether the ptype is in the dictionary. If not, create it
            # and add the prob p and weight w
            if ptype not in pdict:
                #pdict[ptype] = {"p": p, "pw": p*w, "pdivw": p/w, "n": 1} BEFORE
                pdict[ptype] = {"p": p, "pw": 1, "pdivw" : 1, "n": 1}
            else:
                pdict[ptype]["p"] += p
                pdict[ptype]["pw"] += p*w
                pdict[ptype]["pdivw"] += p/w
                pdict[ptype]["n"] += 1
    return pdict

def grow_path(path, extdict, N):
    """Grows a path by N steps, returns all possibilities."""
    paths = [path]
    # path is for example 3-LMR, it can become 3-LMR;4-LML or 3-LMR;4-LMR
    for i in range(N):
        newpaths = []
        for path in paths:
            for ext_path in extdict[path[-1]]:
                newpaths.append(path + [ext_path])
        paths = newpaths
    return paths


# work_dir = "./shuffleboard_reppextis6"
# os.chdir(work_dir)

# intfs = [0.125, 0.175, 0.225, 0.275, 0.325,
#          0.375, 0.425, 0.475, 0.525, 0.575,
#          0.625, 0.675, 0.725, 0.775, 0.825,
#          0.875]
intfs = np.linspace(0.5,8.5,10)*.1
# Next = 6


In [ ]:
from potential import RectangularGridWithRuggedPotential
potke = RectangularGridWithRuggedPotential()

In [ ]:
crossdata = {}
os.getcwd()

In [ ]:
import time
crossdata = {}
NlNextpairs = [(1,1), (1,2), (1,3), (4,1), (4,2), (4,3), (4,4)]
for elid, el in enumerate(NlNextpairs):
    Nl = el[0]
    Next = el[1]
    time.sleep(1)
    work_dir = f"/home/wouter/onesnake/rugged_reppextis{Next}-{Nl}"
    os.chdir(work_dir)

    dictlist = []
    for i in range((len(intfs)+2)):
        dictlist.append(pathens_to_dict(f"pathensemble{i}.txt"))

    for i, el in enumerate(dictlist):
        print(f"{i}: {len(el)}")

    # Let's build a dummy dictionary with all the possible states
    # Each ensemble normally has 4 states. For ensemble i, this is:
    # i-LML, i-LMR, i-RML, i-RMR
    # This is true for i=2, 3, 4, 5, ..., N-2
    # For ensemble 0, we ONLY have 0-RMR
    # For ensemble 1, we ONLY have 1-LML, 1-LMR, 1-RML
    # For ensemble N-1, we ONLY have (N-1)-LMR, (N-1)-RML, (N-1)-RMR
    # For ensemble N, we ONLY have N-LML
    Nintf = len(intfs)
    Nens = Nintf + 2
    ens_paths = {i: [] for i in range(Nens)}
    # first we build the allowable paths for each ensemble
    for i in range(Nens):
        if i == 0:
            ens_paths[i] = [f"{i}-RMR"]
        elif i == 1:
            ens_paths[i] = [f"{i}-LML", f"{i}-LMR", f"{i}-RML"]
        elif i == Nens-2:
            ens_paths[i] = [f"{i}-LMR", f"{i}-RML", f"{i}-RMR"]
        elif i == Nens-1:
            ens_paths[i] = [f"{i}-LML"]
        else:
            ens_paths[i] = [f"{i}-LML", f"{i}-LMR", f"{i}-RML", f"{i}-RMR"]

    pprint(ens_paths)

    # now we will make a second dictionary, with pathtypes as keys, and the 
    # possible extensions as values
    # i-*MR --> i+1-LML and i+1-LMR
    # i-*ML --> i-1-RML and i-1-RMR (if those exist, e.g. 0 will only have 0-RMR)
    extdict = {}
    for iens, ens in enumerate(ens_paths):
        for path in ens_paths[ens]:
            if path[-2:] == "MR":
                extdict[path] = [path for path in ens_paths[iens+1] if path[-3:-1] == "LM"]
            elif path[-2:] == "ML":
                extdict[path] = [path for path in ens_paths[iens-1] if path[-3:-1] == "RM"]
            else:
                print("error")
                break

    pprint(extdict)

    # Now we are ready to build all possible paths that consist of (Next + 1) consecutive states
    # We start by looping through the 
    paths = []
    for enspaths in ens_paths:
        for path in ens_paths[enspaths]:
            paths += grow_path([path], extdict, Next-1)

    T = [[[] for _ in range(len(paths))] for __ in range(len(paths))]


    for i, path1 in enumerate(paths):
        extensions = grow_path(path1, extdict, 1)
        if len(extensions) == 1:
            j = paths.index(extensions[0][1:])
            T[i][j].append((1., 1., 1., iens))
        else:
            assert len(extensions) == 2
            j1 = paths.index(extensions[0][1:])
            j2 = paths.index(extensions[1][1:])
            for iens, ens in enumerate(dictlist):
                toj = [0., 0.]
                toj_n = [0, 0]
                for j, extension in enumerate(extensions):
                    ext_id = ";".join(extension)
                    if ";".join(extension) in ens:
                        toj[j] += ens[ext_id]["p"]
                        toj_n[j] += ens[ext_id]["n"]
                if sum(toj) > 0:
                    crossprob = np.array([x/sum(toj) for x in toj], dtype=np.float128)
                    T[i][j1].append((crossprob[0], sum(toj), sum(toj_n), iens))
                    T[i][j2].append((crossprob[1], sum(toj), sum(toj_n), iens))
                else:
                    pass


    Tmat = np.zeros((len(paths), len(paths)), dtype=np.float128)
    # we take the weighted average of each T[i][j] entry, where w[k] = T[i][j][k][1]
    for i in range(len(paths)):
        for j in range(len(paths)):
            if not T[i][j]:  # if T[i][j] is empty
                Tmat[i, j] = 0.
            else:
                probs = np.array([x[0] for x in T[i][j]], dtype=np.float128)
                weights = np.array([x[1] for x in T[i][j]], dtype=np.float128)
                weights = weights/sum(weights)
                Tmat[i, j] = np.dot(probs, weights)


    # sums of rows should be 1
    for i in range(len(paths)):
        if not np.isclose(np.sum(Tmat[i, :]), 1., atol=1e-10):
            print(f"WARNING: row {i} does not sum to 1, but to {np.sum(Tmat[i, :])}")
            print(f"This is state {paths[i]}")
            if np.sum(Tmat[:,i]) == 66666:
                print("But it is not reachable, so we don't care")
            else:
            # first check the states it can extend to
                extensions = grow_path(paths[i], extdict, 1)
                if len(extensions) == 1:
                    j = paths.index(extensions[0][1:])
                    Tmat[i,j] = 1.
                    print("It had one extension, so we added it")
                    
                else:
                    assert len(extensions) == 2
                    # check whether the other one
                    j1 = paths.index(extensions[0][1:])
                    j2 = paths.index(extensions[1][1:])
                    print(f"Extensions:\n\trow{j1}:{paths[j1]}\n\trow{j2}:{paths[j2]}")
                    if True:
                        print("Setting to 1/2 prob")
                        Tmat[i,j1] = 0.5
                        Tmat[i,j2] = 0.5
                    if False: 
                        #print("Setting the one ending with L to 1, the other to 0")
                        if extensions[0][1][-1] == "L":
                            print(f"Setting {paths[j1]} to 1,\n\tand {paths[j2]} to 0")
                            Tmat[i,j1] = 1.
                            Tmat[i,j2] = 0.
                        else:
                            print(f"Setting {paths[j1]} to 0,\n\tand {paths[j2]} to 1")
                            Tmat[i,j1] = 0.
                            Tmat[i,j2] = 1.

    # Okay, let's just normalize the rows now
    for i in range(len(paths)):
        Tmat[i, :] = Tmat[i, :]/np.sum(Tmat[i, :])

    print("MSM STUFF")
    print(is_transition_matrix(Tmat))
    print(is_connected(Tmat))
    print(is_reversible(Tmat))

    cc_directed = connected_sets(Tmat)
    print(cc_directed)

    # Let's remove the disconnected states from Tmat (remove columns and rows), and then renormalize Tmat
    # we will also have to delete the entries from paths, or we'll mess up indexing
    to_delete = []
    for dset in cc_directed[1:]:
        for el in dset:
            to_delete.append(el)
    to_delete = np.sort(to_delete)

    for row in to_delete[::-1]:
        Tmat = np.delete(Tmat, row, axis=0)
        Tmat = np.delete(Tmat, row, axis=1)
        paths.pop(row)
    for row in Tmat:
        row = row/np.sum(row)

    print(is_transition_matrix(Tmat))
    print(is_connected(Tmat))
    print(is_reversible(Tmat))


    # let's define state A and state B
    # state A are those paths that end with 1-RML or 1-LML 
    # state S are those paths that end with 0-RMR (we are interested in paths starting here)
    # state B are those paths that end with N-1-LMR
    A = [i for i, path in enumerate(paths) if path[-1] == "1-RML" or path[-1] == "1-LML"]
    S = [i for i, path in enumerate(paths) if path[-1] == "0-RMR"]
    B = [i for i, path in enumerate(paths) if path[-1] == f"{Nens-2}-LMR"]
    print("states A:\n\t"+ "\n\t".join([str(paths[i]) + f" id: {i}" for i in A]))
    print("states S:\n\t"+ "\n\t".join([str(paths[i]) + f" id: {i}" for i in S]))
    print("states B:\n\t"+ "\n\t".join([str(paths[i]) + f" id: {i}" for i in B]))

    test = tpt(Tmat, A, B)
    # stationary distribution 
    pi = stationary_distribution(Tmat)
    crossprob = np.sum([pi[i] * test.committor[i]/np.sum([pi[j] for j in S]) for i in S])
    print(crossprob)

    # let's look at committor probabilities up to interface i
    # states S and A remain the same, but state B changes to paths ending with i-LMR
    Pcross = [1.,]
    for iens in range(1, Nens-2):
        B = [i for i, path in enumerate(paths) if path[-1] == f"{iens}-LMR"]
        test = tpt(Tmat, A, B)
        pcross = np.sum([pi[i] * test.committor[i]/np.sum([pi[j] for j in S]) for i in S])
        Pcross.append(pcross)
    print("Cross-probs:\n", Pcross)

    crossdata[elid] = Pcross

In [ ]:
0.075-0.075/4

In [ ]:
# let's read in the RETIS result as well
pretis = []
#with open("../maze2Dretis/report/total-probability.txt", "r") as f:
with open("../rugged_reppextisretis/report/total-probability.txt", "r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        pretis.append([float(el) for el in line.split()])
pretis = np.array(pretis)
prepptis = []
with open("../rugged_reppextis0-1/report/total-probability.txt", "r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        prepptis.append([float(el) for el in line.split()][1])
prepptis = np.array(prepptis)


In [ ]:
# extract pretis value where pretis[x][0] is equal to intfs[1],
# find the index of the closest value to intfs[1]
idx = (np.abs(pretis[:,0] - intfs[1])).argmin()
pretisval = pretis[idx][1]
print(pretisval)
valerrp = pretisval*0.6
valerrm = pretisval*1.4

In [ ]:

import logging

import numpy as np
from simulation import Simulation


%matplotlib qt
%load_ext autoreload

In [ ]:
# Set-up the simulation keys for RETIS
retisset = {"interfaces" : intfs.tolist(),
            "simtype" : "retis", 
            "method" : "load",
            "max_len" : 1000000,
            "dt": 0.002,
            "temperature": .05,
            "friction": 25.,
            "max_cycles": 100000,
            "p_shoot": 0.75,
            "include_stateB": False,
            'prime_both_starts': False,
            'snake_Lmax': 0,
            'max_paths': 1,
            'Nl': 1,
             'dim' : 2,
             'mass' : .1,
}
# or for REPPTIS
repptisset = {"interfaces" : intfs,
             "simtype" : "repptis", 
             "method" : "load",
             "max_len" : 1000000,
             "dt": 0.002,
             "temperature": .05,
             "friction": 25.,
             "max_cycles": 250000,
             "p_shoot": 1.,
             "include_stateB": True,
             'prime_both_starts': True,
             'snake_Lmax': 0,
             'max_paths': 2,
             'sL' : 9,
             'Nl': 1,
             'Next' : 3,
             'yoot' : False, # to set choice = 0 and W = 1.
             'endpoints_only' : False,
             'flipboth': True,
             'binomial': True,
             'random_shoots': True,
             'invert_W' : False,
             'dim' : 2,
             'mass' : .1,
}

%autoreload
logger = logging.getLogger()
file_handler = logging.FileHandler("logging.log")
formatter = logging.Formatter('[%(levelname)s] %(name)s %(funcName)s %(lineno)d: %(message)s')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
logger.setLevel(logging.INFO)
logger.info("Hello world!")
logger.propagate = False
sim = Simulation(repptisset)
xpot, ypot, vpot = sim.ensembles[0].engine.potential.get_potential_plot(N=250)

# sim = extSimulation(repptisset)
# xpot, ypot, vpot = sim.ensembles[0][0].engine.potential.get_potential_plot()


# set loglevel to warning for matplotlib

logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [ ]:
xpot, ypot, vpot = potke.get_potential_plot(N=100)

In [ ]:
pretismax = pretis[-1,1]*1.38
pretismin = pretis[-1,1]/1.38
ppptismax = prepptis[-1]*1.28
ppptismin = prepptis[-1]/1.28

In [ ]:
fig, ax = plt.subplots(figsize=(6,6))


# plot over this ax, but don't twin it (it uses separate x)

# remove xy labels and ticks for ax2
# don't show the ticks on the left 

ax.plot(pretis[:,0], pretis[:,1], lw=3, c="k", label="RETIS")
# ax.plot([intfs[1], intfs[1]], [valerrm, valerrp],lw=2,ls="--", c="k")
ax.plot(intfs, prepptis, marker="x", lw=1,c="k", ms=10, label="PPTIS")
for intf in intfs:
    ax.axvline(intf, lw=0.5, ls="--", c="k")
for elid, pairs in enumerate(NlNextpairs):
    if elid == 6:
        continue
    if elid >= 3:
        markerke = "o" 
    else:
        markerke = "x"
    ax.plot(intfs, crossdata[elid], marker=markerke, lw=1, ms=10, label=fr"{pairs[1]}-{pairs[0]}")
#ax.plot(intfs, pcross10, marker="o", lw=1, ms=10, label=r"$N_{\mathrm{ext}}=10$")
ax.plot([intfs[-1]+0.021, intfs[-1]+0.021], [pretismin, pretismax], lw=2, ls="-", marker="_", c="k")
ax.plot([intfs[-1]+0.021, intfs[-1]+0.021], [ppptismin, ppptismax], lw=2, ls="-", marker="_", c="k")

ax.set_xlabel(r"$\lambda$", fontsize=15)
ax.set_ylabel(r"$P(\lambda_i|\lambda_A)$", fontsize=15)
ax.set_yscale("log")
# ticks should be big as well, also the tickmarks (- and |)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.tick_params(axis='both', which='minor', labelsize=15)
ax2 = ax.twinx()
f = ax2.contourf(ypot, xpot, vpot[::-1,::-1], levels=np.arange(-.15, 0.31, 0.05), cmap="Greys", alpha=0.15)
# legend box has white backghround no alpha so white background
# place it right from the plot area (outside)
ax.legend(fontsize=14,  ncol=2, frameon=True, fancybox=True, edgecolor="black", loc="upper right", bbox_to_anchor=(0.9, 0.9))
ax2.set_yticks([])
fig.tight_layout()
fig.show()
fig.savefig("ruggedresultnew.pdf")

In [ ]:
xpot, ypot, vpot = potke.get_potential_plot(N=250)

In [ ]:
os.getcwd()

In [ ]:
# journal settings for pyplot parameters
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 15
plt.rcParams['legend.title_fontsize'] = 15
plt.rcParams['figure.titlesize'] = 15

# plot potential
fig, ax = plt.subplots(figsize=(8,3))
# contourlines at 0.1, 0.2, 0.3, ..., 0.9
for i,intf in enumerate(intfs):
    ax.axvline(intf*10, lw=1, ls="-", c="k")
    # text above the line with its value 
    # ax.text(intf*10, 4, f"{intf*10:.1f}", fontsize=15, ha="center")  # Adjust y_pos for vertical spacing
    ax.text(intf*10, 3.5, rf"$\lambda_{{{i}}}$", fontsize=15, ha="center")  # Adjust y_pos for vertical spacing
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")
# equal aspect ratio
ax.set_aspect("equal")
#black and white color scheme in matplotlib
f = ax.contourf(10*ypot, 10*xpot, vpot[::-1,::-1], levels=np.arange(-.15, 0.51, 0.05), cmap="Blues")
# make the colorbar small 
fig.colorbar(f, ax=ax, ticks=[-0.15, 0.5],shrink=0.8)
fig.tight_layout(pad=0)
fig.show()
fig.savefig("potential.pdf")

In [ ]:
pcross10 = np.load("crossdata.npy")

In [ ]:

fig, ax = plt.subplots()
for intf in intfs:
    ax.axvline(intf, c="k", lw=0.5, ls="--")
ax.plot(intfs, Pcross, marker="x", c="r", lw=1, ms=5)
ax.set_xlabel(r"$\lambda$")
ax.set_ylabel(r"$P(\lambda_i|\lambda_A)$")
ax.set_yscale("log")
fig.show()

In [ ]:
#Pcrosslist = []2.26641383241116e-05

Pcrosslist.append(Pcross)

In [ ]:
# let's read in the RETIS result as well
pretis = []
#with open("../maze2Dretis/report/total-probability.txt", "r") as f:
with open("../shuffleboard_retis/report/total-probability.txt", "r") as f:

    for line in f:
        if line.startswith("#"):
            continue
        pretis.append([float(el) for el in line.split()])
pretis = np.array(pretis)



In [ ]:
prepptis = []
with open("../shuffleboard_repptis/report/total-probability.txt", "r") as f:
    for line in f:
        if line.startswith("#"):
            continue
        prepptis.append([float(el) for el in line.split()])
prepptis = np.array(prepptis)

In [ ]:
fig, ax = plt.subplots()
for intf in intfs:
    ax.axvline(intf, lw=0.5, ls="--", c="k")


ax.plot(prepptis[:,0], prepptis[:,1], lw=1, c="b", label=r"$M=1$")#label="PPTIS")
for i, pcross in enumerate(Pcrosslist):
    ax.plot(intfs, pcross, marker="o", lw=1, ms=10, label=fr"$M=${i+2}")

ax.plot(pretis[:,0], pretis[:,1], lw=1, c="k", label="RETIS")
ax.set_xlabel(r"$\lambda$", fontsize=15)
ax.set_ylabel(r"$P(\lambda_i|\lambda_A)$", fontsize=15)
ax.set_yscale("log")
# ticks should be big as well, also the tickmarks (- and |)
ax.tick_params(axis='both', which='major', labelsize=15)
ax.tick_params(axis='both', which='minor', labelsize=15)
ax.legend(fontsize=15)
fig.tight_layout()
fig.show()
fig.savefig("shuffleresult.svg")
fig.savefig("shuffleresult.png", dpi=300)
fig.savefig("shuffleresult.pdf")